# Исследование надежности заемщиков

На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

## Изучение общей информации о данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data['total_income'].describe

<bound method NDFrame.describe of 0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21402, dtype: float64>

In [15]:
data['total_income'] = data['total_income'].astype(int)

In [16]:
data['total_income'].describe

<bound method NDFrame.describe of 0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21402, dtype: int32>

### Обработка дубликатов

In [17]:
data['education'] = data['education'].str.lower()

In [18]:
data.duplicated().sum()

71

In [19]:
data = data.drop_duplicates()

### Категоризация данных

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [22]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [24]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
data_children_grouped = data.groupby('children')['debt'].agg(['sum','count','mean']).reset_index()
data_children_grouped = data_children_grouped.rename(columns={'children':'Количество детей','sum':'Количество должников' \
                                                              ,'count':'Количество заемщиков','mean':'Доля должников'})
data_children_grouped = data_children_grouped.loc[:2]
data_children_grouped

,Количество детей,Количество должников,Количество заемщиков,Доля должников
0,0,1063,14091,0.075438
1,1,444,4808,0.092346
2,2,194,2052,0.094542


In [26]:
# Разница долей должников между группами с детьми и без
children_debt = data_children_grouped.loc[1:2,'Количество должников'].sum() \
/ data_children_grouped.loc[1:2,'Количество заемщиков'].sum() 
wochildren_debt = data_children_grouped.loc[0,'Доля должников']
having_children_difference = children_debt - wochildren_debt
having_children_difference

0.01756469247268845

**Вывод:** Была рассчитана доля должников среди заемщиков без детей, с одним ребенком и двумя детьми. Из-за малого количества заемщиков имеющих 3х или более детей - данными о них было решено пренебречь. Между группами с одним и двумя детьми сильных различий по доле задержек по выплатам нет. Заемщики не имеющие детей на 1.7% реже имеют задержки по платежам, чем заемщики с детьми. Эти данные можно использовать при построении модели кредитного скоринга.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
data_family_grouped = data.groupby('family_status')['debt'].agg(['sum','count','mean']).reset_index()
data_family_grouped = data_family_grouped.rename(columns={'family_status':'Семейное положение','sum':'Количество должников', \
                                                          'count':'Количество заемщиков','mean':'Доля должников'}).loc[[0,3,4]]
data_family_grouped.sort_values('Доля должников')

,Семейное положение,Количество должников,Количество заемщиков,Доля должников
4,женат / замужем,927,12261,0.075606
3,гражданский брак,385,4134,0.093130
0,Не женат / не замужем,273,2796,0.097639


**Вывод:** Наибольший процент задержек по выплатам в категориях "гражданский брак" и "не женат/не замужем" - более 9,3% в каждой. У заемщиков, состоящих в официальном браке 7,5% задержек. Прослеживается точная зависимость процента задолженностей от семейного положения. Эти данные определенно следует использовать при построении модели кредитного скоринга.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
data_income_grouped = data.groupby('total_income_category')['debt'].agg(['sum','count','mean']).reset_index()
data_income_grouped = data_income_grouped.rename(columns={'total_income_category':'Ежемесячный доход', \
                                                          'sum':'Количество должников','count':'Количество заемщиков', \
                                                          'mean':'Доля должников'}).loc[1:2]
data_income_grouped['Ежемесячный доход'] = ['200001-1000000','50001-200000']
data_income_grouped

,Ежемесячный доход,Количество должников,Количество заемщиков,Доля должников
1,200001-1000000,354,5014,0.070602
2,50001-200000,1353,15921,0.084982


In [29]:
# Разница долей должников между группами по доходу
data_income_difference = data_income_grouped.loc[2,'Доля должников'] - data_income_grouped.loc[1,'Доля должников']
data_income_difference

0.014379785592239222

**Вывод:** Была рассчитана доля должников для групп заемщиков c зарплатами в интервалах 50001-200000рублей и 200001-1000000рублей. Остальными зарпалатными группами было решено пренебречь в рассчетах, ввиду малого количества заемщиков в них. Можно заметить тенденцию, что люди с месячным доходом 200001-1000000рублей на 1.4% реже имеют задержки по выплатам, чем люди с месячным доходом 50001-200000рублей, что можно учесть при построении модели кредитного скоринга.

### Как разные цели кредита влияют на его возврат в срок?

In [30]:
data_purpose_grouped = data.groupby('purpose_category')['debt'].agg(['sum','count','mean']).reset_index()
data_purpose_grouped = data_purpose_grouped.rename(columns={'purpose_category':'Цель кредита','sum':'Количество должников', \
                                                            'count':'Количество заемщиков','mean':'Доля должников'})
data_purpose_grouped.sort_values('Доля должников')

,Цель кредита,Количество должников,Количество заемщиков,Доля должников
1,операции с недвижимостью,780,10751,0.072551
3,проведение свадьбы,183,2313,0.079118
2,получение образования,369,3988,0.092528
0,операции с автомобилем,400,4279,0.093480


**Вывод:** Среди групп заемщиков по целевым категориям наибольший процент задержек по платежам в группах "получение образования" и "операции с автомобилем", порядка 9.3% в каждой. На 1.4% меньше процент задержек в категории "проведение свадьбы", а именно 7,9%. Самые платежеспособные в данной выборке - заемщики, берущие кредит на "операции с недвижимостью", они задерживают платежи в 7.3% случаев, что на 2% меньше, чем у лидеров по задержкам платежей. Эти данные определенно стоит использовать при построении модели кредитного скоринга.

### Приведите возможные причины появления пропусков в исходных данных.

**Ответ:** Пропуски в столбцах 'days_employed' и 'total_income' встречались в одних и тех же строках. Возможно была какая-то техническая проблема во время сбора данных. Например, из-за отсутствия данных в одном из этих столбцов данные другого столбца так же не попадали в таблицу. Или, человек заполнявший данные о заемщиках мог указывать данные в некорректном формате, из-за чего стало невозможным их преобразование в числовое значение.

### Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

**Ответ:** Некоторые данные в столбцах с количественными переменными могут сильно отличаться от других, что может исказить результаты вычислений, если заполнить пропуски в столбце средним арифметическим значением. Поэтому, медиана - лучшее решение для заполнения пропусков количественных переменных, она находится в середине выборки значений и очень большие/маленькие значения оказывают на нее меньшее влияние.

## Общий вывод

В данном исследовании были изучены зависимости семейного положения, количества детей, уровня дохода и целей кредита на возврат кредитов в срок.

Семейное положение имеет влияние на возврат кредита в срок. Заемщики находящиеся/находившиеся в официальном браке значительно реже имеют просрочки по платежам.

Наличие детей влияет на возврат кредита в срок. Однако, на процент просрочек по платежам не оказывает конкретное влияние количество детей у заемщика.

Уровень дохода влияет на возврат кредита в срок. Заемщики с ежемесячным доходом выше 200000рублей чаще выплачивают кредит вовремя. 

Цель кредита имеет влияние на возврат кредита в срок. Наименьший процент задержек в оплате среди покупателей недвижимости. Немногим больше процент задержек в оплате среди молодоженов. Самые большой процент задержек в оплате у тех, кто берет кредит на авто/образование.

Все эти данные можно учитывать при построении модели кредитного скоринга.